In [73]:
from datamodule import Datamodule
from models.cnn8 import CNN8simple
from models.mlp import MLPsimple

from pathlib import Path
import numpy as np

import torch

In [78]:
params = {}
params['BFD_enc'] = {'batch_norm': True,'batch_size': 16,'dropout': 0.4,'lr': 0.001,'negative_slope': 0.05}
params['BFD_gray'] = {'batch_norm': False,'batch_size': 16,'dropout': 0.4,'lr': 0.001,'negative_slope': 0.1}
params['BFD_rgb'] = {'batch_norm': False,'batch_size': 8,'dropout': 0.0,'lr': 0.00005,'negative_slope': 0.02}
params['ChaLearn_enc'] = {'batch_norm': False,'batch_size': 4,'dropout': 0.3,'lr': 0.001,'negative_slope': 0.1}
params['ChaLearn_gray'] = {'batch_norm': True,'batch_size': 4,'dropout': 0.0,'lr': 0.001,'negative_slope': 0.01}
params['ChaLearn_rgb'] = {'batch_norm': False,'batch_size': 8,'dropout': 0.0,'lr': 0.00005,'negative_slope': 0.1}

In [79]:
for model_name, value in params.items():
    
    dataset_name = model_name.split('_')[0]
    dataset_version = model_name.split('_')[1]
    
    model = torch.load(f'model/k_cross/{model_name}.pt')
    model.eval()
    
    test_data_path = Path(f'data/DENTA/{dataset_name}/{dataset_version}/test/')
    dm = Datamodule(batch_size=41, val_dir=test_data_path)
    dm.setup(val_only=True)
    print(model_name)
    
    for batch in dm.val_dataloader():
        norm_data = 'original' if dataset_version=='enc' else 'normalized'
        X, Y = batch[norm_data], batch['label']
        Y_pred = model(X.cuda())
        Y_pred = Y_pred.cpu()
        Y_pred = np.where(Y_pred > 0.0, 1, 0)
        acc_class = np.sum(Y_pred == np.array(Y), axis=0) / len(Y)
        
        Y_pred_1 = np.ones_like(Y)
        acc_class_1 = np.sum(Y_pred_1 == np.array(Y), axis=0) / len(Y)

    print('Gdy same 1')
    print(acc_class_1)
    print(acc_class_1.mean())

    print('Predykcja:')
    print(acc_class)
    print(acc_class.mean())
    print('----------------------------------------------------------')
    print()

file: data/DENTA/BFD/enc/test/test.pickle
BFD_enc
Gdy same 1
[0.65853659 0.7804878  0.75609756 0.29268293 0.70731707]
0.6390243902439025
Predykcja:
[0.6097561  0.7804878  0.73170732 0.29268293 0.65853659]
0.6146341463414634
----------------------------------------------------------

file: data/DENTA/BFD/gray/test/test.pickle
BFD_gray
Gdy same 1
[0.65853659 0.7804878  0.75609756 0.29268293 0.70731707]
0.6390243902439025
Predykcja:
[0.65853659 0.7804878  0.75609756 0.56097561 0.58536585]
0.6682926829268293
----------------------------------------------------------

file: data/DENTA/BFD/rgb/test/test.pickle
BFD_rgb
Gdy same 1
[0.65853659 0.7804878  0.75609756 0.29268293 0.70731707]
0.6390243902439025
Predykcja:
[0.63414634 0.80487805 0.75609756 0.34146341 0.75609756]
0.6585365853658537
----------------------------------------------------------

file: data/DENTA/ChaLearn/enc/test/test.pickle
ChaLearn_enc
Gdy same 1
[0.65853659 0.7804878  0.75609756 0.29268293 0.70731707]
0.6390243902439025